
# Trabalho Prático 2 - Matteo Freitas Reis



# Configuração


Realiza o download da legislação por meio de um arquivo de texto público hospedado no Google Drive.

In [1]:
import requests

# ID do arquivo no Google Drive (extraído da URL pública)
file_id = '1LbkNw-iusTdGrNh0WFl9tHER_0Cg9pa8'

# Cria o link de download direto
url = f'https://drive.google.com/uc?export=download&id={file_id}'

# Faz a requisição
response = requests.get(url)

# Verifica se deu certo
if response.status_code == 200:
    texto = response.text
    print(texto[:1000])  # Mostra os primeiros 1000 caracteres
else:
    print("Erro ao acessar o arquivo:", response.status_code)





        Art. 1º A Previdência Social, mediante contribuição, tem por fim assegurar aos seus beneficiários meios indispensáveis de manutenção, por motivo de incapacidade, desemprego involuntário, idade avançada, tempo de serviço, encargos familiares e prisão ou morte daqueles de quem dependiam economicamente.

        Art. 2º A Previdência Social rege-se pelos seguintes princípios e objetivos:

        I - universalidade de participação nos planos previdenciários;

        II - uniformidade e equivalência dos benefícios e serviços às populações urbanas e rurais;

        III - seletividade e distributividade na prestação dos benefícios;

        IV - cálculo dos benefícios considerando-se os salários-de-contribuição corrigidos monetariamente;

        V - irredutibilidade do valor dos benefícios de forma a preservar-lhes o poder aquisitivo;

        VI - valor da renda mensal dos benefícios substitutos do salário-de-contribuição ou do rendimento do trabalho do segurado não inferior ao

Define uma função chamada dividir_artigos, que recebe um texto completo contendo os artigos legais e o divide em uma lista, onde cada elemento corresponde ao conteúdo de um artigo da lei. Utilizando expressões regulares, a função localiza as marcações de artigo (como "Art. 1º" ou "Art. 10.") e extrai os trechos de texto entre essas marcações. Durante o processo, a função ignora automaticamente os artigos completamente vetados (revogados), cujos números são definidos no conjunto artigos_revogados. Isso garante que apenas os artigos ativos sejam incluídos na lista final.
(o modo que a lista de artigos vetados foi definida está no final no notebook).

In [2]:
import re

def dividir_artigos(texto):
    artigos_texto = []
    pos = 0
    numero_artigo = 1

    #completamente vetados
    artigos_revogados = {7, 8, 30, 64, 79, 81, 82, 83, 84, 85, 87, 95, 100, 123, 127, 139, 140, 141, 144, 145, 146, 147, 148, 151, 152}

    while True:
        if numero_artigo in artigos_revogados:
            # Pula os artigos revogados, não procura o artigo no texto, apenas incrementa o número
            numero_artigo += 1
            continue

        if numero_artigo <= 9:
            padrao = re.compile(rf'(Art\. {numero_artigo}º)')
        else:
            padrao = re.compile(rf'(Art\. {numero_artigo}\.)')

        resultado = padrao.search(texto, pos)
        if not resultado:
            # Se não encontrar o próximo artigo, pegamos o restante do texto como último artigo (se houver)
            if pos < len(texto):
                artigos_texto.append(texto[pos:].strip())
            break

        start = resultado.start()
        # Se não for o primeiro artigo, o artigo anterior vai até start do atual
        if numero_artigo > 1:
            artigos_texto.append(texto[pos:start].strip())

        pos = start
        numero_artigo += 1

    return artigos_texto

# Exemplo de uso (use o texto do exemplo anterior)
lista_artigos = dividir_artigos(texto)



Configura a chave de API

In [11]:
import os
from google.colab import userdata

try:
    os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
    print("Chave de API do Gemini carregada dos Secrets.")
except KeyError:
    print("Erro: A chave 'GOOGLE_API_KEY' não foi encontrada nos Secrets.")

import google.generativeai as genai  # Importa a biblioteca generativeai como genai
genai.configure(api_key=os.environ.get("GOOGLE_API_KEY"))

Chave de API do Gemini carregada dos Secrets.


Define as funções para realização de chamadas de prompts

In [4]:
def chamada_LLM(prompt, model_name='gemini-2.0-flash', temperature=0.7, max_output_tokens=500):
  try:
    model = genai.GenerativeModel(model_name)
    response = model.generate_content(
        prompt,
        generation_config={
            "temperature": temperature,
            "max_output_tokens": max_output_tokens
        }
    )
    return response.text
  except Exception as e:
    print(f"Erro ao gerar resposta com o modelo {model_name}: {e}")
    return None

def prompt_simples(instrucao, contexto, dados_entrada, indicador_saida):

  prompt = f"""
  Instrução: {instrucao}

  Contexto: {contexto}

  Dados de entrada: {dados_entrada}

  Indicador de saída: {indicador_saida}
  """
  return chamada_LLM(prompt)

# Determina os artigos relevantes para a pergunta


O código apresentado define a função identificar_artigos_uteis, cujo objetivo é identificar quais artigos legais são potencialmente relevantes para responder a uma pergunta jurídica específica. A função divide uma lista de artigos em blocos menores e, para cada bloco, realiza múltiplas consultas a uma função chamada prompt_simples. Essa função avalia a relevância dos artigos com base em uma pergunta e retorna uma resposta simples ("Sim" ou "Não"). A estrutura do código permite aplicar a técnica de votação majoritária para determinar, com mais robustez, se um determinado bloco de artigos é útil.

O código implementa a técnica de validação auto-consistente, ao realizar múltiplas iterações (por padrão, três) para cada bloco de artigos e depois verificar qual foi a resposta mais frequente. Em vez de confiar em uma única resposta do modelo, o sistema avalia a consistência das respostas ao repetir a mesma pergunta diversas vezes e contabilizar as ocorrências de "Sim". Se a maioria das respostas for afirmativa, o bloco de artigos é considerado útil.

In [5]:
import time

def identificar_artigos_uteis(pergunta, lista_artigos, prompt_simples, num_artigos_prompt=5, numero_iteracoes=3, tempo_espera=3):
    """
    Identifica os artigos potencialmente úteis com base em uma pergunta jurídica, usando votação majoritária.

    Parâmetros:
    - pergunta: string contendo a pergunta jurídica.
    - lista_artigos: lista de strings, cada uma representando um artigo legal.
    - prompt_simples: função que gera respostas com base em instruções e contexto.
    - num_artigos_prompt: número de artigos por bloco a serem avaliados em conjunto.
    - numero_iteracoes: número de votações por bloco.
    - tempo_espera: tempo de espera entre cada chamada ao prompt (em segundos).

    Retorna:
    - artigos_uteis: string contendo os artigos considerados úteis, separados por "\n\n".
    - respostas_por_bloco: lista de listas com as respostas obtidas para cada bloco.
    """

    instrucao_base = "Diga se algum dos artigos abaixo tem alguma relação com a pergunta: " + pergunta
    contexto = "Você é um verificador de relações entre artigos e perguntas jurídicas da previdência social."
    indicador_saida = "Responda únicamente Sim ou Não."

    respostas_por_bloco = []
    artigos_uteis = ""

    # Agrupar os artigos em blocos
    for i in range(0, len(lista_artigos), num_artigos_prompt):
        bloco_artigos = lista_artigos[i:i+num_artigos_prompt]
        texto_bloco = "\n\n".join(bloco_artigos)
        respostas_do_bloco = []

        print(f"Bloco de artigos {i+1} a {i+len(bloco_artigos)}")

        for _ in range(numero_iteracoes):
            resposta = prompt_simples(instrucao_base, contexto, texto_bloco, indicador_saida)
            print("Resposta:", resposta)
            if resposta:
                respostas_do_bloco.append(resposta.lower().strip())
            else:
                respostas_do_bloco.append("erro ou resposta vazia")
            time.sleep(tempo_espera)

        respostas_por_bloco.append(respostas_do_bloco)

        # Verifica se a maioria respondeu "sim"
        votos_sim = sum(1 for r in respostas_do_bloco if "sim" in r)
        if votos_sim > numero_iteracoes // 2:
            artigos_uteis += texto_bloco + "\n\n"

    return artigos_uteis.strip(), respostas_por_bloco


# Responde consulta

O código implementa duas funções principais: gerar_uma_cadeia e validar_auto_consistente, que juntas compõem um sistema de raciocínio jurídico automatizado para responder perguntas com base nos artigos legais fornecidos. A função gerar_uma_cadeia realiza uma cadeia de prompts encadeados (prompt chaining), enquanto a função validar_auto_consistente realiza múltiplas execuções dessa cadeia para aplicar a técnica de validação auto-consistente. Ao final, a resposta considerada mais confiável é selecionada com base na consistência das respostas anteriores.

Na função gerar_uma_cadeia, temos o uso de prompt chaining em quatro etapas sequenciais. Cada etapa realiza uma tarefa diferente, mas interdependente: (1) decompor a pergunta jurídica em subquestões, (2) identificar os requisitos legais com base nos artigos úteis e nas subquestões, (3) formular uma resposta com justificativa jurídica, e (4) extrair as referências legais citadas. A saída de cada etapa alimenta a próxima, permitindo uma construção lógica e gradual do raciocínio, como seria feito por um especialista humano.

Além do encadeamento de prompts, o código também aplica a técnica de Chain-of-Thought (CoT) na primeira etapa da cadeia. CoT é um método que estimula o modelo a "pensar em voz alta", decompondo problemas complexos em etapas mais simples antes de responder diretamente. Isso é feito ao instruir o modelo a "decompor a pergunta em subquestões numeradas", o que melhora a clareza do raciocínio e favorece decisões mais embasadas nas etapas posteriores.

A função validar_auto_consistente aplica o conceito de validação auto-consistente (Self-Consistency) ao executar várias cadeias de raciocínio completas (num_cadeias, geralmente 3) para a mesma pergunta. Cada cadeia gera uma resposta independente, e o sistema analisa se todas as respostas convergem para a mesma conclusão (sim, não, depende). Se houver convergência, a resposta é considerada confiável; caso contrário, o sistema sinaliza ambiguidade. Isso torna a resposta final mais robusta, pois reduz a influência de variabilidade ou ruído na geração de texto do modelo.

Por fim, havendo ou não divergência entre as cadeias, o código executa um último prompt para sintetizar todas as respostas geradas e produzir uma justificativa final. Esse passo, além de ser parte do prompt chaining, também atua como uma forma de meta-análise, utilizando os próprios resultados anteriores para justificar a melhor decisão. Essa abordagem imita uma forma sofisticada de tomada de decisão com base em múltiplas opiniões, aproximando-se de um painel de especialistas. Assim, o código integra as três técnicas avançadas — prompt chaining, chain-of-thought e validação auto-consistente — de forma complementar e eficaz.

In [6]:
import time

def gerar_uma_cadeia(pergunta, artigos_uteis, contexto, tempo_espera=3):
    # Etapa 1 - Decompor a pergunta (Chain-of-Thought)
    instrucao1 = f"Decomponha a seguinte pergunta em subquestões ou etapas lógicas: {pergunta}"
    indicador_saida1 = "Liste as subquestões numeradas."
    subquestoes = prompt_simples(instrucao1, contexto, artigos_uteis, indicador_saida1)
    time.sleep(tempo_espera)

    # Etapa 2 - Identificar os requisitos legais aplicáveis
    instrucao2 = (
        "Com base nos artigos fornecidos, indique os requisitos legais e hipóteses relevantes para cada subquestão."
    )
    indicador_saida2 = "Explique os requisitos legais relacionados às subquestões."
    requisitos = prompt_simples(instrucao2, contexto + "\n\nSubquestões:\n" + subquestoes, artigos_uteis, indicador_saida2)
    time.sleep(tempo_espera)

    # Etapa 3 - Responder a pergunta com justificativa
    instrucao3 = "Com base nos requisitos legais e nos artigos apresentados, responda à pergunta original."
    indicador_saida3 = "Diga se sim, se não ou se depende. Em seguida, justifique em português com base jurídica."
    resposta_final = prompt_simples(instrucao3, contexto, artigos_uteis + "\n\nRequisitos:\n" + requisitos, indicador_saida3)
    time.sleep(tempo_espera)

    # Etapa 4 - Extrair referências precisas
    instrucao4 = "Liste os artigos, incisos ou parágrafos exatos da legislação que foram utilizados na justificativa acima."
    indicador_saida4 = "Forneça a lista de referências jurídicas citadas."
    referencias = prompt_simples(instrucao4, contexto, artigos_uteis + "\n\nResposta:\n" + resposta_final, indicador_saida4)

    return {
        "resposta": resposta_final,
        "referencias": referencias.strip()
    }


def validar_auto_consistente(pergunta, artigos_uteis, num_cadeias=3, tempo_espera=3):
    contexto = "Você é um advogado especialista em Direito Previdenciário."

    respostas = []
    classificacoes = []
    referencias_lista = []

    for i in range(num_cadeias):
        print(f"Gerando cadeia {i+1}...")
        resultado = gerar_uma_cadeia(pergunta, artigos_uteis, contexto, tempo_espera=tempo_espera)
        resposta = resultado["resposta"]
        referencias = resultado["referencias"]

        print(f"Resposta {i+1}:\n{resposta}\n")
        print(f"Referências {i+1}:\n{referencias}\n")

        respostas.append(resposta)
        referencias_lista.append(referencias)

        # Classificação da resposta
        r_lower = resposta.lower()
        if "depende" in r_lower:
            classificacoes.append("depende")
        elif "não" in r_lower or "nao" in r_lower:
            classificacoes.append("não")
        elif "sim" in r_lower:
            classificacoes.append("sim")
        else:
            classificacoes.append("indefinido")

        time.sleep(tempo_espera)

    # Verificar consistência das classificações
    unica_classificacao = set(classificacoes)
    if len(unica_classificacao) == 1:
        resultado_final = f"As {num_cadeias} cadeias concordam com a resposta: '{classificacoes[0]}'."
        justificativa_final = "Não foram encontradas ambiguidades significativas entre as respostas geradas."
    else:
        resultado_final = "As respostas geradas apresentam divergência, indicando ambiguidade na questão."
        justificativa_final = (
            f"Classificações encontradas: {classificacoes}. "
            "Recomenda-se análise mais detalhada para resolver a ambiguidade."
        )

    # Etapa de escolha final
    instrucao_final = (
        "Baseado nas múltiplas respostas acima e considerando suas divergências, "
        "justifique qual resposta deve ser considerada mais confiável e explique as ambiguidades."
    )
    indicador_saida_final = "Resposta em português para a pergunta " + pergunta + ", clara e detalhada."
    contexto_final = contexto + "\n\nRespostas:\n" + "\n\n".join(respostas) + "\n\n" + resultado_final

    justificativa = prompt_simples(instrucao_final, contexto_final, artigos_uteis, indicador_saida_final)

    return {
        "respostas": respostas,
        "classificacoes": classificacoes,
        "referencias": referencias_lista,
        "resultado_final": resultado_final,
        "justificativa_final": justificativa,
    }


# Exemplos de Perguntas-Caso

In [12]:
pergunta = "João, segurado do INSS, ficou incapacitado para o trabalho por 18 dias. Ele tem direito ao auxílio-doença? Quando começa o pagamento?"

print("Definição dos artigos úteis para responder à pergunta")
artigos_uteis, respostas = identificar_artigos_uteis(pergunta, lista_artigos, prompt_simples, num_artigos_prompt=10, numero_iteracoes=3, tempo_espera=3)

print("\nRespondendo à pergunta usando CoT, Prompt Chaining e Validação Auto-Consistente...")
resultado = validar_auto_consistente(pergunta, artigos_uteis, num_cadeias=3, tempo_espera=4)

print("\n### Resumo da Validação Auto-Consistente ###")
for i, (resposta, referencia) in enumerate(zip(resultado["respostas"], resultado["referencias"]), 1):
    print(f"\n--- Cadeia {i} ---")
    print("Resposta:", resposta)
    print("Referências jurídicas utilizadas:", referencia)

print("\nClassificações:", resultado["classificacoes"])
print("\nResultado final:", resultado["resultado_final"])
print("\nJustificativa final:")
print(resultado["justificativa_final"])


Definição dos artigos úteis para responder à pergunta
Bloco de artigos 1 a 10
Resposta: Sim

Resposta: Sim

Resposta: Sim

Bloco de artigos 11 a 20
Resposta: Não

Resposta: Não.

Resposta: Não.

Bloco de artigos 21 a 30
Resposta: Não

Resposta: Não

Resposta: Não

Bloco de artigos 31 a 40
Resposta: Sim

Resposta: Sim

Resposta: Sim

Bloco de artigos 41 a 50
Resposta: Não

Resposta: Não

Resposta: Não

Bloco de artigos 51 a 60
Resposta: Sim

Resposta: Sim

Resposta: Sim

Bloco de artigos 61 a 70
Resposta: Não

Resposta: Não

Resposta: Não

Bloco de artigos 71 a 80
Resposta: Não

Resposta: Não

Resposta: Não

Bloco de artigos 81 a 90
Resposta: Não

Resposta: Não

Resposta: Não

Bloco de artigos 91 a 100
Resposta: Não

Resposta: Não

Resposta: Não

Bloco de artigos 101 a 110
Resposta: Não

Resposta: Não

Resposta: Não

Bloco de artigos 111 a 120
Resposta: Não

Resposta: Não

Resposta: Não

Bloco de artigos 121 a 130
Resposta: Não

Resposta: Não

Resposta: Não

Bloco de artigos 131 a 131
R

In [13]:
pergunta = "Ana estava recebendo auxílio-doença, recuperou-se e voltou ao trabalho. Trinta dias depois, voltou a se afastar pelo mesmo motivo. O benefício pode ser restabelecido?"

print("Definição dos artigos úteis para responder à pergunta")
artigos_uteis, respostas = identificar_artigos_uteis(pergunta, lista_artigos, prompt_simples, num_artigos_prompt=10, numero_iteracoes=3, tempo_espera=4)

print("\nRespondendo à pergunta usando CoT, Prompt Chaining e Validação Auto-Consistente...")
resultado = validar_auto_consistente(pergunta, artigos_uteis, num_cadeias=3, tempo_espera=4)

print("\n### Resumo da Validação Auto-Consistente ###")
for i, (resposta, referencia) in enumerate(zip(resultado["respostas"], resultado["referencias"]), 1):
    print(f"\n--- Cadeia {i} ---")
    print("Resposta:", resposta)
    print("Referências jurídicas utilizadas:", referencia)

print("\nClassificações:", resultado["classificacoes"])
print("\nResultado final:", resultado["resultado_final"])
print("\nJustificativa final:")
print(resultado["justificativa_final"])


Definição dos artigos úteis para responder à pergunta
Bloco de artigos 1 a 10
Resposta: Não

Resposta: Não

Resposta: Não

Bloco de artigos 11 a 20
Resposta: Sim

Resposta: Sim

Resposta: Sim

Bloco de artigos 21 a 30
Resposta: Sim

Resposta: Sim

Resposta: Sim

Bloco de artigos 31 a 40
Resposta: Não.

Resposta: Não.

Resposta: Não

Bloco de artigos 41 a 50
Resposta: Não

Resposta: Não

Resposta: Não

Bloco de artigos 51 a 60
Resposta: Sim

Resposta: Sim

Resposta: Sim

Bloco de artigos 61 a 70
Resposta: Não

Resposta: Não

Resposta: Não

Bloco de artigos 71 a 80
Resposta: Não

Resposta: Não

Resposta: Não

Bloco de artigos 81 a 90
Resposta: Não.

Resposta: Não

Resposta: Não.

Bloco de artigos 91 a 100
Resposta: Não

Resposta: Não

Resposta: Não

Bloco de artigos 101 a 110
Resposta: Não

Resposta: Não

Resposta: Não

Bloco de artigos 111 a 120
Resposta: Não

Resposta: Não

Resposta: Não

Bloco de artigos 121 a 130
Resposta: Não

Resposta: Não

Resposta: Não

Bloco de artigos 131 a 131

In [14]:
pergunta = "Carlos precisou se afastar por 12 dias por motivo de doença. Ele terá direito ao auxílio-doença?"

print("Definição dos artigos úteis para responder à pergunta")
artigos_uteis, respostas = identificar_artigos_uteis(pergunta, lista_artigos, prompt_simples, num_artigos_prompt=10, numero_iteracoes=3, tempo_espera=4)

print("\nRespondendo à pergunta usando CoT, Prompt Chaining e Validação Auto-Consistente...")
resultado = validar_auto_consistente(pergunta, artigos_uteis, num_cadeias=3, tempo_espera=4)

print("\n### Resumo da Validação Auto-Consistente ###")
for i, (resposta, referencia) in enumerate(zip(resultado["respostas"], resultado["referencias"]), 1):
    print(f"\n--- Cadeia {i} ---")
    print("Resposta:", resposta)
    print("Referências jurídicas utilizadas:", referencia)

print("\nClassificações:", resultado["classificacoes"])
print("\nResultado final:", resultado["resultado_final"])
print("\nJustificativa final:")
print(resultado["justificativa_final"])


Definição dos artigos úteis para responder à pergunta
Bloco de artigos 1 a 10
Resposta: Sim

Resposta: Sim

Resposta: Sim

Bloco de artigos 11 a 20
Resposta: Sim

Resposta: Sim

Resposta: Sim

Bloco de artigos 21 a 30
Resposta: Sim

Resposta: Sim

Resposta: Sim

Bloco de artigos 31 a 40
Resposta: Não

Resposta: Não

Resposta: Não

Bloco de artigos 41 a 50
Resposta: Não

Resposta: Não

Resposta: Não

Bloco de artigos 51 a 60
Resposta: Sim

Resposta: Sim

Resposta: Sim

Bloco de artigos 61 a 70
Resposta: Não

Resposta: Não.

Resposta: Não.

Bloco de artigos 71 a 80
Resposta: Não

Resposta: Não.

Resposta: Não

Bloco de artigos 81 a 90
Resposta: Sim

Resposta: Sim

Resposta: Sim

Bloco de artigos 91 a 100
Resposta: Não

Resposta: Não

Resposta: Não

Bloco de artigos 101 a 110
Resposta: Não.

Resposta: Não.

Resposta: Não.

Bloco de artigos 111 a 120
Resposta: Não.

Resposta: Não.

Resposta: Não.

Bloco de artigos 121 a 130
Resposta: Sim

Resposta: Sim

Resposta: Sim

Bloco de artigos 131 

In [15]:
pergunta = "Pedro contribuiu por 24 meses e foi considerado incapaz permanentemente para qualquer trabalho. Quais são os requisitos para aposentadoria por invalidez e ele se enquadra?"

print("Definição dos artigos úteis para responder à pergunta")
artigos_uteis, respostas = identificar_artigos_uteis(pergunta, lista_artigos, prompt_simples, num_artigos_prompt=10, numero_iteracoes=3, tempo_espera=4)

print("\nRespondendo à pergunta usando CoT, Prompt Chaining e Validação Auto-Consistente...")
resultado = validar_auto_consistente(pergunta, artigos_uteis, num_cadeias=3, tempo_espera=4)

print("\n### Resumo da Validação Auto-Consistente ###")
for i, (resposta, referencia) in enumerate(zip(resultado["respostas"], resultado["referencias"]), 1):
    print(f"\n--- Cadeia {i} ---")
    print("Resposta:", resposta)
    print("Referências jurídicas utilizadas:", referencia)

print("\nClassificações:", resultado["classificacoes"])
print("\nResultado final:", resultado["resultado_final"])
print("\nJustificativa final:")
print(resultado["justificativa_final"])


Definição dos artigos úteis para responder à pergunta
Bloco de artigos 1 a 10
Resposta: Sim

Resposta: Sim

Resposta: Sim

Bloco de artigos 11 a 20
Resposta: Sim

Resposta: Sim

Resposta: Sim

Bloco de artigos 21 a 30
Resposta: Sim

Resposta: Sim

Resposta: Sim

Bloco de artigos 31 a 40
Resposta: Sim

Resposta: Sim

Resposta: Sim

Bloco de artigos 41 a 50
Resposta: Sim

Resposta: Sim

Resposta: Sim

Bloco de artigos 51 a 60
Resposta: Não

Resposta: Não

Resposta: Não

Bloco de artigos 61 a 70
Resposta: Não

Resposta: Não

Resposta: Não

Bloco de artigos 71 a 80
Resposta: Não

Resposta: Não

Resposta: Não

Bloco de artigos 81 a 90
Resposta: Não

Resposta: Não

Resposta: Não

Bloco de artigos 91 a 100
Resposta: Não

Resposta: Não

Resposta: Não

Bloco de artigos 101 a 110
Resposta: Não

Resposta: Não

Resposta: Não

Bloco de artigos 111 a 120
Resposta: Não

Resposta: Não

Resposta: Não

Bloco de artigos 121 a 130
Resposta: Sim

Resposta: Sim

Resposta: Sim

Bloco de artigos 131 a 131
Res

# Verificação quais artigos estavam vetados.
Para que a execução dos prompts fosse otimizada, tambem foi feita uma etapa de Prompt-Based Filtering, que utilizou um prompt em cada artigo para perguntar se o mesmo estava vetado ou ainda estava vigente.

 Como não é determinístico, também foi feita uma leitura humana do texto original para confirmar se os indicados como vetados pelo prompt realmente eram vetados. Foi assim que a lista de artigos revogados foi criada.

In [ ]:
import re

def dividir_artigos_com_vetados(texto):
    artigos_texto = []
    pos = 0
    numero_artigo = 1

    #completamente vetados
    artigos_revogados = {82, 83, 84, 85, 144, 145, 146, 147, 152}

    while True:
        if numero_artigo in artigos_revogados:
            # Pula os artigos revogados, não procura o artigo no texto, apenas incrementa o número
            numero_artigo += 1
            continue

        if numero_artigo <= 9:
            padrao = re.compile(rf'(Art\. {numero_artigo}º)')
        else:
            padrao = re.compile(rf'(Art\. {numero_artigo}\.)')

        resultado = padrao.search(texto, pos)
        if not resultado:
            # Se não encontrar o próximo artigo, pegamos o restante do texto como último artigo (se houver)
            if pos < len(texto):
                artigos_texto.append(texto[pos:].strip())
            break

        start = resultado.start()
        # Se não for o primeiro artigo, o artigo anterior vai até start do atual
        if numero_artigo > 1:
            artigos_texto.append(texto[pos:start].strip())

        pos = start
        numero_artigo += 1

    return artigos_texto

# Exemplo de uso (use o texto do exemplo anterior)
lista_artigos_total = dividir_artigos_com_vetados(texto)


In [ ]:
import time

instrucao = "diga se o arigo abaixo está vetado ou não com base somente no texto do próprio artigo. escreva vetado somente \
para os artigos completamente vetados, se for só um subtópico não vetado, escreva, vigente"
contexto = "Você é um verificador se o artigo está vetado ou não"
indicador_saida = "responda únicamente vetado ou vigente"

for artigo in lista_artigos_total:
  resposta = prompt_simples(instrucao, contexto, artigo, indicador_saida)
  print(artigo[:10])
  if resposta:
    print(resposta)
  time.sleep(3)